In [2]:
import pandas as pd
import numpy as np
import igraph

In [13]:
### Loading Raw Data
df = {}
with open('./amzn_reviews/reviews_Health_and_Personal_Care_5.json', 'r') as f:
    for i, line in enumerate(f.readlines()):
        df[i] = eval(line)

raw_data = pd.DataFrame.from_dict(df, orient='index')

# fixing the helpful is a list issue
raw_data['helpful_ct'] = raw_data.helpful.apply(lambda x : x[0])
raw_data['unhelpful_ct'] = raw_data.helpful.apply(lambda x : x[1] - x[0])
raw_data.drop('helpful', axis=1, inplace=True)

# subset most popular products (> 50 reviews)
asin_list = list(raw_data.groupby(['asin']).count().sort_values('reviewerID',ascending=False).query('reviewerID > 50').index)

In [18]:
# subset users with most purchases (> 8 purchases)
usr_list = list(raw_data.groupby('reviewerID').\
                                                count().\
                                                sort_values('asin',ascending=False).\
                                                query('asin > 8').index)

In [20]:
raw_data[raw_data['reviewerID'].isin(usr_list)]

,index,reviewerID,asin,reviewerName,reviewText,overall,summary,unixReviewTime,reviewTime,helpful_ct,unhelpful_ct
5,85,A3F66CWNNM3OD,B00000J47L,Andrew,I've been using these for years for my Xbox 36...,5.0,Been using these for years,1387584000,"12 21, 2013",1,0
9,89,A21OBYW0N5ENS7,B00000J47L,"Bassocantor "" OracleMagician""",Very good experience with this brand over the ...,4.0,&#10004; Lasts on All-day Hikes,1400025600,"05 14, 2014",1,0
12,92,A1N655X9X7C6QY,B00000J47L,"C. Cook ""LIVE..........LOVE........AND..........",Right out of the box and ready to go !Everythi...,5.0,Out of the box ready to go................,1357948800,"01 12, 2013",0,0
17,97,AEL6CQNQXONBX,B00000J47L,Cute Chihuahua,In this day and age it is hard to understand w...,5.0,SAVE LOTS OF MONEY AND THE ENVIRONMENT AT THE ...,1227571200,"11 25, 2008",1,0
28,108,A2032LF6FWWK8E,B00000J47L,Jay Riemenschneider,"At 2500ah, these rechargeable NiMh batteries p...",5.0,"Excellent power, quality NiMh batteries to go ...",1141689600,"03 7, 2006",6,4
29,109,A5GPH59NDWJRB,B00000J47L,Jenna of the Jungle,"I just gave up and threw out these batteries, ...",1.0,Boo! Dead batteries!,1253232000,"09 18, 2009",2,0
40,120,AOEAD7DPLZE53,B00000J47L,NLee the Engineer,This product is advertised as the NEW Energize...,2.0,"""Everything Old Is New Again"" featuring the En...",1290211200,"11 20, 2010",11,0
42,122,ALDAF4VVLFRHP,B00000J47L,Patricia Brouillette,I originally got this for my daughter's leapfr...,5.0,"Great idea, sames money",1327622400,"01 27, 2012",0,0
43,123,A3SNJORZ5SMZZ9,B00000J47L,PeachPecan,I was going through quite a few batteries for ...,5.0,Just right!,1309824000,"07 5, 2011",1,0
44,124,ATZJNEXX7LWD3,B00000J47L,RG,"As many other reviewers have pointed out, thes...",2.0,Not worth it! Self-discharge rate is too high!,1190592000,"09 24, 2007",6,1


In [190]:
# saving the file based on product filter
#raw_data[raw_data['asin'].isin(asin_list)].reset_index().to_feather('./amzn_popular_product_stats.feather')

# saving the file based on user filter
#raw_data[raw_data['reviewerID'].isin(usr_list)].reset_index().to_feather('./amzn_active_users_stats.feather')

# reading the file
#raw_data = pd.read_feather('./amzn_popular_product_stats.feather')

active_users = pd.read_feather('./amzn_active_users_stats.feather')

In [191]:
active_users.head()

,level_0,index,reviewerID,asin,reviewerName,reviewText,overall,summary,unixReviewTime,reviewTime,helpful_ct,unhelpful_ct
0,5,85,A3F66CWNNM3OD,B00000J47L,Andrew,I've been using these for years for my Xbox 36...,5.0,Been using these for years,1387584000,"12 21, 2013",1,0
1,9,89,A21OBYW0N5ENS7,B00000J47L,"Bassocantor "" OracleMagician""",Very good experience with this brand over the ...,4.0,&#10004; Lasts on All-day Hikes,1400025600,"05 14, 2014",1,0
2,12,92,A1N655X9X7C6QY,B00000J47L,"C. Cook ""LIVE..........LOVE........AND..........",Right out of the box and ready to go !Everythi...,5.0,Out of the box ready to go................,1357948800,"01 12, 2013",0,0
3,17,97,AEL6CQNQXONBX,B00000J47L,Cute Chihuahua,In this day and age it is hard to understand w...,5.0,SAVE LOTS OF MONEY AND THE ENVIRONMENT AT THE ...,1227571200,"11 25, 2008",1,0
4,28,108,A2032LF6FWWK8E,B00000J47L,Jay Riemenschneider,"At 2500ah, these rechargeable NiMh batteries p...",5.0,"Excellent power, quality NiMh batteries to go ...",1141689600,"03 7, 2006",6,4


In [192]:
active_users = active_users.query('overall <= 2.0')

## Getting Pairwise Connections per User

In [80]:
import itertools

In [146]:
active_users.groupby('asin').reviewerID.apply(lambda x : list(x))

['A5GPH59NDWJRB', 'AOEAD7DPLZE53', 'ATZJNEXX7LWD3', 'A2Y3WWPUKIJ59I']

In [81]:
def product_to_user_pair(df):
    prod_user_set = df.groupby('asin').reviewerID.apply(lambda x : set(x))
    return {prod : list(itertools.combinations(users,2)) for prod,users in prod_user_set.items()}

In [125]:
prod_user_pairs = product_to_user_pair(active_users)

In [126]:
user_pairs = set()

In [127]:
for prod, pairs in prod_user_pairs.items():
    for pair in pairs:
        user_pairs.add(pair)

## With Integers Instead of Strings

In [141]:
users = set(active_users.reviewerID)

user_to_int = dict(zip(users, np.arange(0,len(users))))

nodes = [user_to_int[user] for user in users]

edges = [(user_to_int[t[0]], user_to_int[t[1]]) for t in user_pairs]

## With Strings Instead of Integers

In [144]:
pd.DataFrame({
              'node1':[t[0] for t in edges],
              'node2':[t[1] for t in edges]
             }
            ).\
to_feather('./edge_list_int.feather')

## Node List

In [ ]:
pd.DataFrame({,
              'node_id':,
              'num_reviews':,
              'num_'
    
    
})

In [157]:
active_users.head()

,level_0,index,reviewerID,asin,reviewerName,reviewText,overall,summary,unixReviewTime,reviewTime,helpful_ct,unhelpful_ct,reviewerID_int
5,29,109,A5GPH59NDWJRB,B00000J47L,Jenna of the Jungle,"I just gave up and threw out these batteries, ...",1.0,Boo! Dead batteries!,1253232000,"09 18, 2009",2,0,448
6,40,120,AOEAD7DPLZE53,B00000J47L,NLee the Engineer,This product is advertised as the NEW Energize...,2.0,"""Everything Old Is New Again"" featuring the En...",1290211200,"11 20, 2010",11,0,148
9,44,124,ATZJNEXX7LWD3,B00000J47L,RG,"As many other reviewers have pointed out, thes...",2.0,Not worth it! Self-discharge rate is too high!,1190592000,"09 24, 2007",6,1,294
11,51,131,A2Y3WWPUKIJ59I,B00000J47L,Veil_Lord,While these batteries have a higher mAh rating...,1.0,Good Capacity/Bad Charge Retention,1261612800,"12 24, 2009",2,0,1215
101,598,748,A26LD9FQHTM8ZH,B00004OCLJ,OlyNomad,There aren't enough bristles and they are too ...,2.0,Would not buy again,1399939200,"05 13, 2014",0,0,508


In [195]:
active_users['node_id'] = active_users.reviewerID.apply(lambda x : user_to_int[x])

In [196]:
active_users['review_len'] = active_users.reviewText.apply(lambda x : len(x.split()))

In [161]:
nodes = pd.DataFrame({
                      'node_id':np.arange(1359)
                     })

In [203]:
nodes = nodes.merge(active_users.groupby('node_id').review_len.mean().to_frame(), 
                       how='left',
                       left_on='node_id',
                       right_index=True
                )

nodes = nodes.merge(active_users.groupby('node_id').node_id.count().to_frame(), 
                       how='left',
                       left_on='node_id',
                       right_index=True
                )


In [193]:
active_users

,level_0,index,reviewerID,asin,reviewerName,reviewText,overall,summary,unixReviewTime,reviewTime,helpful_ct,unhelpful_ct
5,29,109,A5GPH59NDWJRB,B00000J47L,Jenna of the Jungle,"I just gave up and threw out these batteries, ...",1.0,Boo! Dead batteries!,1253232000,"09 18, 2009",2,0
6,40,120,AOEAD7DPLZE53,B00000J47L,NLee the Engineer,This product is advertised as the NEW Energize...,2.0,"""Everything Old Is New Again"" featuring the En...",1290211200,"11 20, 2010",11,0
9,44,124,ATZJNEXX7LWD3,B00000J47L,RG,"As many other reviewers have pointed out, thes...",2.0,Not worth it! Self-discharge rate is too high!,1190592000,"09 24, 2007",6,1
11,51,131,A2Y3WWPUKIJ59I,B00000J47L,Veil_Lord,While these batteries have a higher mAh rating...,1.0,Good Capacity/Bad Charge Retention,1261612800,"12 24, 2009",2,0
101,598,748,A26LD9FQHTM8ZH,B00004OCLJ,OlyNomad,There aren't enough bristles and they are too ...,2.0,Would not buy again,1399939200,"05 13, 2014",0,0
125,826,1277,A100WO06OQR8BQ,B000050FDY,"Duffer. ""Bo""",Much easier (and much less costly) to just rem...,1.0,Not Worth The Price,1207267200,"04 4, 2008",2,5
179,1204,1943,A1YUL9PCJR3JTY,B000052XMR,"O. Brown ""Ms. O. Khannah-Brown""",*This band-aid is the worst band-aid I've ever...,1.0,Practically Useless BandAid,1295568000,"01 21, 2011",1,1
202,1365,2470,A3VLTT45T236B,B000052YHR,W. Taylor,Bought both the Mach 3 and Mach 3 Turbo blades...,1.0,Horrible Quality Decline!!!!!!!!,1392163200,"02 12, 2014",0,2
203,1387,2573,A2V1J3JT5OOZFO,B000052YOB,Dragonwort Castle,I think this has an ingredient I have an aller...,2.0,Probably OK for Many,1397520000,"04 15, 2014",0,0
235,1699,3711,A2C27IQUH9N1Z,B000053L2J,audrey,I received a free sample power bar through the...,2.0,"good taste, horrible texture",1216944000,"07 25, 2008",0,0


In [205]:
nodes.columns = ['node_id', 'avg_len', 'num_reviews']

In [207]:
nodes.to_feather('./node_attr.feather')

In [211]:
nodes

,node_id,avg_len,num_reviews
0,0,131.000000,1
1,1,38.250000,4
2,2,136.000000,1
3,3,161.000000,1
4,4,66.000000,1
5,5,165.600000,5
6,6,58.000000,4
7,7,173.000000,1
8,8,259.000000,2
9,9,114.500000,2
